In [9]:
import os
import numpy as np
import pandas as pd 
from tqdm import tqdm
import pathlib 

def check_normalized(features):
    print(features.shape)
    norms = np.linalg.norm(features, axis=1)
    true_or_false = np.allclose(norms, 1.0)
    print(f"feature_shape: {features.shape} normalized to 1 {true_or_false}")
    return true_or_false

img_npz_file = r"D:\work_space\work\austin_paper_coding\austin_paper1\feature_output\movie_img_features_20251103_1457.npz"

img_data = np.load(img_npz_file, allow_pickle=True)

print(img_data.files)

text_file = r"D:\work_space\work\austin_paper_coding\austin_paper1\feature_output\movie_text_features_20251103_1457.npz"

text_data = np.load(text_file, allow_pickle=True)

check_normalized(img_data["features"])
check_normalized(text_data["features"])


### construct train / val /test

print(os.getcwd())
RATING_PATH = r"..\data\ratings.dat"


#score >= 4 is like the movie 
GOOD_RATING = 4.0
TRAIN_THRESHOLD = 5  # at least 5 ratings for each user to be included
TRAIN_RATIO = 0.8
VAL_RATIO = TEST_RATIO = (1-TRAIN_RATIO)/2

SAVE_PATH  = pathlib.Path("../data/train_val_test_dir")
pathlib.Path.mkdir(SAVE_PATH, exist_ok=True)

ratings = pd.read_csv(
    RATING_PATH,
    sep="::",
    names=["user_id", "movie_id", "rating", "timestamp"],
    engine="python",
).sort_values(by=["user_id", "timestamp"]).reset_index(drop=True)

print(ratings["rating"][:2])
print(type(ratings["rating"]))
## create a new labe in ratings 
ratings["label"] = (ratings["rating"] >= GOOD_RATING).astype(int)



#####
print("############################################")
user_count = ratings["user_id"].value_counts()
print(user_count[:10])
active_usrs = user_count[user_count >= 5].index
ratings = ratings[ratings["user_id"].isin(active_usrs)].sort_values(by=["user_id", "timestamp"]).reset_index(drop=True)
print(ratings[:4])

train_list ,val_list ,test_list = [],[],[]

for uid ,df in tqdm(ratings.groupby("user_id")):
    n = len(df)
    if n < TRAIN_THRESHOLD :
        train_list.append(df)
    else :
        train_end = int(n * TRAIN_RATIO)
        val_end = int (n * (TRAIN_RATIO + VAL_RATIO))
        train_list.append(df[:train_end])
        val_list.append(df[train_end:val_end])
        test_list.append(df[val_end:])
        
keep_cols = ['user_id', 'movie_id', 'rating', 'timestamp', 'label']
      
        
train_df = pd.concat(train_list)[keep_cols].reset_index(drop=True)

val_df = pd.concat(val_list)[keep_cols].reset_index(drop=True)

test_df = pd.concat(test_list)[keep_cols].reset_index(drop=True)

train_df.to_csv(SAVE_PATH / "train.csv",index=False)
val_df.to_csv(SAVE_PATH / "val.csv",index=False)
test_df.to_csv(SAVE_PATH / "test.csv",index=False)


['ids', 'features']
(3883, 512)
feature_shape: (3883, 512) normalized to 1 True
(3883, 384)
feature_shape: (3883, 384) normalized to 1 True
d:\work_space\work\austin_paper_coding\austin_paper1\scritps
0    4
1    5
Name: rating, dtype: int64
<class 'pandas.core.series.Series'>
############################################
user_id
4169    2314
1680    1850
4277    1743
1941    1595
1181    1521
889     1518
3618    1344
2063    1323
1150    1302
1015    1286
Name: count, dtype: int64
   user_id  movie_id  rating  timestamp  label
0        1      3186       4  978300019      1
1        1      1270       5  978300055      1
2        1      1721       4  978300055      1
3        1      1022       5  978300055      1


100%|██████████| 6040/6040 [00:00<00:00, 22026.88it/s]


In [6]:
##############
import numpy as np  
img_npz_file = r"D:\work_space\work\austin_paper_coding\austin_paper1\feature_output\movie_img_features_20251103_1457.npz"
npg_features = np.load(img_npz_file, allow_pickle=True)
print(npg_features["arr_0"])
print(npg_features["arr_1"])



KeyError: 'arr_0 is not a file in the archive'